In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)

LR_tune,N_value = np.meshgrid(lr_tune,n_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)

lrn_tune = np.hstack((LR_tune,N_value))

In [6]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [7]:
u0 = 50.0
loss_thresh = 0.1

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [8]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True

        
        self.n = torch.tensor(n_val)
        
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
nan_tune = []
for tune_reps in range(25):
    label = "inv_HT_atanh_tune" + str(tune_reps)
    max_reps = 10 #10
    max_iter = 75#75

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []

    lambda1_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    n_val = lrn_tune[tune_reps,1]

    for reps in range(max_reps):
        print(label)
        'Generate Training data'
        print(reps)
        torch.manual_seed(reps*36)
        
        train_loss = []
        test_mse_loss = []
        test_re_loss = []   
        alpha_val = []

        lambda1_val = []

        N_f = 50000 #Total number of collocation points 
        N_train = 500

        layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        
        PINN = Sequentialmodel(layers,n_val)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrn_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')
        
        
        nan_flag = train_model(max_iter,reps)

        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        alpha_full.append(alpha_val)
  
        lambda1_full.append(lambda1_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,"lambda1": lambda1_full, "label": label}
    savemat(label+'.mat', mdic) 


inv_HT_atanh_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1019.8298 Test MSE 1027.0576375692217 Test RE 0.5394939501351783 Lambda1 -0.07625545
1 Train Loss 838.1357 Test MSE 858.0963389696396 Test RE 0.493125198271195 Lambda1 -0.09567029
2 Train Loss 838.0638 Tes

0 Train Loss 845.22107 Test MSE 862.862424861552 Test RE 0.4944927734874957 Lambda1 -0.05587272
1 Train Loss 838.07074 Test MSE 858.2029185294875 Test RE 0.49315582154292914 Lambda1 -0.05805434
2 Train Loss 838.06305 Test MSE 858.2586413675491 Test RE 0.4931718315070859 Lambda1 -0.05811001
3 Train Loss 838.06274 Test MSE 858.2653777015986 Test RE 0.4931737669164847 Lambda1 -0.058116116
4 Train Loss 838.0627 Test MSE 858.2681441190239 Test RE 0.49317456173090846 Lambda1 -0.058118556
5 Train Loss 838.06256 Test MSE 858.2705123014642 Test RE 0.49317524212813585 Lambda1 -0.058120593
6 Train Loss 838.06244 Test MSE 858.2727898920677 Test RE 0.49317589649673893 Lambda1 -0.058122486
7 Train Loss 838.0624 Test MSE 858.2747872853777 Test RE 0.4931764703618751 Lambda1 -0.058124106
8 Train Loss 838.0624 Test MSE 858.2768170835732 Test RE 0.4931770535364789 Lambda1 -0.058125667
9 Train Loss 838.0623 Test MSE 858.2787134749854 Test RE 0.49317759838179787 Lambda1 -0.05812711
10 Train Loss 838.06226 

2 Train Loss 838.0627 Test MSE 858.2432990924809 Test RE 0.4931674235050312 Lambda1 -0.19786496
3 Train Loss 838.0609 Test MSE 858.2715229394138 Test RE 0.4931755324919907 Lambda1 -0.19795375
4 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
5 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
6 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
7 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
8 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
9 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
10 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
11 Train Loss 838.06067 Test MSE 858.2745885977633 Test RE 0.493176413277557 Lambda1 -0.19796425
12 Train Loss 838.06067 Test MSE 858.2

4 Train Loss 838.06244 Test MSE 858.2690620763886 Test RE 0.49317482546730784 Lambda1 -0.050259367
5 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
6 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
7 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
8 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
9 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
10 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
11 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
12 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
13 Train Loss 838.0623 Test MSE 858.2712850625802 Test RE 0.49317546414820956 Lambda1 -0.050260857
14 Train Loss 8

4 Train Loss 838.063 Test MSE 858.2686104900381 Test RE 0.49317469572300104 Lambda1 -0.057985228
5 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
6 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
7 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
8 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
9 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
10 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
11 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
12 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
13 Train Loss 838.0629 Test MSE 858.2693704897675 Test RE 0.49317491407687525 Lambda1 -0.05798587
14 Train Loss 838.0629 Tes

...

54 Train Loss 453109.75 Test MSE 3563.868330955895 Test RE 1.0049628225826759 Lambda1 -3.0802774e-05
55 Train Loss 448797.56 Test MSE 3563.8976496261807 Test RE 1.0049669563090529 Lambda1 -6.3123814e-05
56 Train Loss 444805.84 Test MSE 3563.9736443284364 Test RE 1.0049776709486136 Lambda1 -7.302364e-05
57 Train Loss 440115.97 Test MSE 3563.971536764992 Test RE 1.0049773738007213 Lambda1 -6.487379e-05
58 Train Loss 435237.25 Test MSE 3563.9785166583106 Test RE 1.0049783579038982 Lambda1 -5.9741084e-05
59 Train Loss 428995.34 Test MSE 3564.0550142083584 Test RE 1.0049891433181966 Lambda1 -6.589682e-05
60 Train Loss 423057.66 Test MSE 3564.140165121388 Test RE 1.0050011486368104 Lambda1 -7.586567e-05
61 Train Loss 418837.56 Test MSE 3564.1817517825452 Test RE 1.0050070118349987 Lambda1 -7.236097e-05
62 Train Loss 413444.03 Test MSE 3564.517705361236 Test RE 1.005054375826215 Lambda1 -7.661463e-05
63 Train Loss 408965.66 Test MSE 3564.736988291028 Test RE 1.0050852899493772 Lambda1 -8.0039

27 Train Loss 225540.88 Test MSE 3514.7237823253768 Test RE 0.9980097190486673 Lambda1 -1.5924295e-05
28 Train Loss 222641.44 Test MSE 3514.6977501750266 Test RE 0.9980060231128309 Lambda1 2.5406132e-05
29 Train Loss 220176.5 Test MSE 3514.6157267943727 Test RE 0.9979943776867733 Lambda1 1.733155e-05
30 Train Loss 218033.83 Test MSE 3514.5861547179693 Test RE 0.9979901791014423 Lambda1 2.1622329e-05
31 Train Loss 215322.6 Test MSE 3514.649204800154 Test RE 0.9979991308069691 Lambda1 2.8073087e-05
32 Train Loss 213268.69 Test MSE 3514.7199965174223 Test RE 0.9980091815563372 Lambda1 5.1887255e-06
33 Train Loss 210858.9 Test MSE 3514.7583059367867 Test RE 0.9980146205455204 Lambda1 1.6209353e-05
34 Train Loss 208002.52 Test MSE 3514.759696328946 Test RE 0.9980148179462296 Lambda1 2.2287684e-05
35 Train Loss 206109.58 Test MSE 3514.8395712875167 Test RE 0.9980261581154344 Lambda1 1.4959856e-05
36 Train Loss 203240.06 Test MSE 3514.724382901694 Test RE 0.9980098043158195 Lambda1 -1.7414388

0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.99
Training time: 2.99
inv_HT_atanh_tune22
9
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
9
0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.88
Training time: 2.88
inv_HT_

0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.89
Training time: 2.89
inv_HT_atanh_tune23
9
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
9
0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.89
Training time: 2.89
inv_HT_

0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.79
Training time: 2.79
inv_HT_atanh_tune24
9
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
9
0 Train Loss nan Test MSE nan Test RE nan Lambda1 nan
NAN BREAK!
Training time: 2.86
Training time: 2.86


In [14]:
import scipy.io as sio

In [18]:
for tune_reps in range(25):
    label = "inv_HT_atanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["lambda1"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   -0.5674061
1   -0.18346347
2   [[nan]]
3   nan
4   [[nan]]
5   -0.22076301
6   -0.08117101
7   [[nan]]
8   nan
9   [[nan]]
10   [[ 2.9659973e-04  3.6526890e-04  3.6491931e-04 -1.7225918e-03
  -2.8505474e-01 -6.6395712e-01 -6.7049617e-01 -1.0269625e+00
  -1.2820612e+00 -1.3224475e+00 -1.2399526e+00 -1.1882937e+00
  -8.2210255e-01 -8.6709797e-01 -8.5426378e-01 -9.1268712e-01
  -9.7879589e-01 -1.0841706e+00 -1.2198769e+00 -1.2949730e+00
  -1.3551496e+00 -1.4492692e+00 -1.4796087e+00 -1.4836943e+00
  -1.5168719e+00 -1.5208520e+00 -1.5149412e+00 -1.5133348e+00
  -1.5150899e+00 -1.5119671e+00 -1.5120102e+00 -1.5249616e+00
  -1.5612912e+00 -1.6117815e+00 -1.6300784e+00 -1.6384304e+00
  -1.6809042e+00 -1.6705692e+00 -1.6539471e+00 -1.6466869e+00
  -1.6284616e+00 -1.5976505e+00 -1.5627764e+00 -1.5062680e+00
  -1.5258645e+00 -1.5533000e+00 -1.5546029e+00 -1.5747427e+00
  -1.5576591e+00 -1.5923307e+00 -1.5989798e+00 -1.6002145e+00
  -1.5846990e+00 -1.5285442e+00 -1.4696697e+00 -1.4721315e+00


In [19]:
lrn_tune[11]

array([0.25, 3.  ])